In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.fftpack import fft
from datetime import datetime



**What This Code Does**  

Filters data to only include one reagent master lot (first one found).  

Plots test results over time to visualize trends.  

Computes and plots a rolling average to detect gradual shifts.  

Performs a Fourier Transform (FFT) to analyze periodic fluctuations. 
 


In [ ]:
file_path = "/Users/abalusawudu/Desktop/Capstone/Capstone-Project/alinity_data_set_1.csv"
df = pd.read_csv(file_path)

df['datetimestamplocal'] = pd.to_datetime(df['Unnamed: 0'])

df = df.dropna(subset=['correctedcount'])

df = df.sort_values(by='datetimestamplocal')

selected_lot = df['reagentmasterlotnumber'].iloc[0]  # Select first available lot
df_filtered = df[df['reagentmasterlotnumber'] == selected_lot]

# Plot test results over time (Filtered by single reagent lot)
plt.figure(figsize=(12, 6))
plt.scatter(df_filtered['datetimestamplocal'], df_filtered['correctedcount'], alpha=0.5, s=10)
plt.xlabel("Timestamp")
plt.ylabel("Test Result (Corrected Count)")
plt.title(f"Test Results Over Time (Reagent Lot: {selected_lot})")
plt.xticks(rotation=45)
plt.grid()
plt.show()

# Compute and plot a rolling average (50-sample window) for filtered data
df_filtered['rolling_avg'] = df_filtered['correctedcount'].rolling(window=50, min_periods=1).mean()

plt.figure(figsize=(12, 6))
plt.plot(df_filtered['datetimestamplocal'], df_filtered['correctedcount'], alpha=0.4, label="Raw Data")
plt.plot(df_filtered['datetimestamplocal'], df_filtered['rolling_avg'], color='red', label="Rolling Average (50 Samples)")
plt.xlabel("Timestamp")
plt.ylabel("Test Result (Corrected Count)")
plt.title(f"Rolling Average of Test Results Over Time (Reagent Lot: {selected_lot})")
plt.xticks(rotation=45)
plt.legend()
plt.grid()
plt.show()

# Perform Fourier Transform to detect periodic trends
df_filtered['time_seconds'] = (df_filtered['datetimestamplocal'] - df_filtered['datetimestamplocal'].min()).dt.total_seconds()

signal_filtered = df_filtered['correctedcount'].values
N_filtered = len(signal_filtered)
freqs_filtered = np.fft.fftfreq(N_filtered, d=(df_filtered['time_seconds'].max() - df_filtered['time_seconds'].min()) / N_filtered)
fft_values_filtered = np.abs(fft(signal_filtered))

# Plot FFT result for filtered data
plt.figure(figsize=(12, 6))
plt.plot(freqs_filtered[:N_filtered // 2], fft_values_filtered[:N_filtered // 2])  # Only positive frequencies
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
plt.title(f"Fourier Transform of Test Results (Reagent Lot: {selected_lot})")
plt.grid()
plt.show()
